# Tải các package

In [1]:
!pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-lyqirsph
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-lyqirsph
  Resolved https://github.com/huggingface/trl.git to commit ea7a1be92c1262b95a57c55a6602a9251ad4afa6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q dataset

In [3]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# Lựa chọn mô hình mistral 7B instruct

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Mistral patching. Transformers = 4.46.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Thêm vào LoRA Adapter và chúng tả chỉ cập nhật từ 1 đến 10% trong số của mô hình

🔥 Tham số r = 8, 16, 32,... với các tác vụ càng phức tạp thì tăng tham số này lên (thường sẽ là đến 128)

🔥 LoRA alpha hường sử dụng 16

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Chuẩn bị dữ liệu

Chúng ta sẽ chọn `chatml` để làm chuẩn finetune. `ChatML` sẽ có định dạng như sau:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE}**: Chúng ta sẽ sử dụng `get_chat_template` để lấy đúng template. Unsolothl hỗ trợ các chuẩn bao gồm : `zephyr`, `chatml`, `mistral`, `llama`, `alpaca`, `vicuna`, `vicuna_old`.\\



In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    map_eos_token = True
)

def formating_prompts_func(examples):
  convos = examples['conversations']
  texts = [tokenizer.apply_chat_template(convo , tokenize = False, add_generation_prompt = False) for convo in convos]
  return {'text' : texts,}
pass

from datasets import load_dataset
dataset = load_dataset('qnguyen3/demo_faq', split = 'train')
dataset = dataset.map(formating_prompts_func, batched = True, )

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [7]:
dataset['text'][0]

'<|im_start|>user\nWhat is VinaLLaMA?<|im_end|>\n<|im_start|>assistant\nVinaLLaMA is an open-source, state-of-the-art Large Language Model for the Vietnamese language.<|im_end|>\n'

# Kiểm tra 5 element để xem chatml hoạt động

In [8]:
dataset[5]['conversations']

[{'from': 'human',
  'value': 'How does VinaLLaMA contribute to the Vietnamese AI landscape?'},
 {'from': 'gpt',
  'value': 'VinaLLaMA marks a significant advancement in the Vietnamese AI landscape and offers a versatile resource for various applications.'}]

In [9]:
dataset[5]['text']

'<|im_start|>user\nHow does VinaLLaMA contribute to the Vietnamese AI landscape?<|im_end|>\n<|im_start|>assistant\nVinaLLaMA marks a significant advancement in the Vietnamese AI landscape and offers a versatile resource for various applications.<|im_end|>\n'

# Nếu muốn tự tạo template riêng cho bản thấn mình

In [10]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% endif %}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
  print('Done')
  tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

# Train model

In [11]:

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Nếu có nhiều GPU, thì nó sẽ hỗ trợ chia các batch cho từng GPU
        gradient_accumulation_steps = 4, # Tích luỹ Gradient cứ 4 step 1 lần
        warmup_steps = 5,
        num_train_epochs = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.547 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,4.362300
2,7.364700
3,8.044400
4,6.212200
5,5.630200
6,2.309900
7,2.868900
8,2.318700
9,1.660300
10,1.538000


In [14]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

142.5106 seconds used for training.
2.38 minutes used for training.
Peak reserved memory = 5.043 GB.
Peak reserved memory for training = 0.496 GB.
Peak reserved memory % of max memory = 34.194 %.
Peak reserved memory for training % of max memory = 3.363 %.


# Inference

In [22]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Can you tell me how many tokens were VinaLlama trained on?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Phải add dòng này vào cho việc GEN
    return_tensors = "pt",
).to("cuda")

print(inputs)

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

tensor([[  523, 28766,   321, 28730,  2521, 28766, 28767,  1838,    13,  6325,
           368,  1912,   528,   910,  1287, 16246,   654,   550,  1380, 28758,
         28714,  2786, 10898,   356, 28804,     2,    13, 28789, 28766,   321,
         28730,  2521, 28766, 28767,   489, 11143,    13]], device='cuda:0')


['<|im_start|>user\nCan you tell me how many tokens were VinaLlama trained on?<|im_end|>\n<|im_start|>assistant\nSure, VinaLLaMA was trained on 1.5 trillion tokens.<|im_end|>']

In [23]:
print(tokenizer.batch_decode(outputs)[0])

<|im_start|>user
Can you tell me how many tokens were VinaLlama trained on?<|im_end|>
<|im_start|>assistant
Sure, VinaLLaMA was trained on 1.5 trillion tokens.<|im_end|>


# Chúng ta có thể dùng text stream để hiện thị các từ GEN ra lần lượt thay vì đợi mô hình GEN xong mới in ra

In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
The next number in the sequence is 13.<|im_end|>


# @SAVING AND LOADING FINETUNE MODEL

In [34]:
from huggingface_hub import notebook_login

notebook_login()

**[NOTE]** : Lưu ý mở chế độ write trong phần `Access Token` để có thể push được model lên hugging-face

In [45]:
model.save_pretrained("unsloth_vinallama") # Local saving
model.push_to_hub("HaoNN/unsloth_vinallama") # Online saving

# Saving to float15 for VLLM

Hỗ trợ lưu trực tiếp `float16`. Chọn `merge_16bit` cho float16 và `merge_4bit` cho int4.

In [46]:
# # Merge to 16bit
# model.save_pretrained_merged("unsloth_model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("HaoNN/unsloth_vinallama_model", tokenizer, save_method = "merged_16bit")

# # Merge to 4bit
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Will remove a cached repo with size 1.2K
100%|██████████| 32/32 [03:26<00:00,  6.44s/it]


In [47]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")